# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
np.random.seed(0)

In [6]:
RANDOM_STATES_ITER = iter(np.random.randint(0, np.iinfo(np.int32).max, size=100000))

In [7]:
# n_init parameter for DeltaSpectralClustering
# a high number should produce more stable final solutions
SC_N_INIT = 50

## Consensus clustering

In [8]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 60

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 60}

In [9]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Load ensemble

In [10]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [11]:
full_ensemble = np.load(output_file)

In [12]:
display(full_ensemble.shape)

(4428, 3749)

# Load ensemble coassociation distance matrix

In [13]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [14]:
ensemble_coassoc_matrix = np.load(output_file)

In [15]:
display(ensemble_coassoc_matrix.shape)

(3749, 3749)

In [16]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.05638587, 0.05412138, ..., 0.59986413, 0.66148571,
        0.66757246],
       [0.05638587, 0.        , 0.02740036, ..., 0.60144928, 0.66011429,
        0.66530797],
       [0.05412138, 0.02740036, 0.        , ..., 0.59805254, 0.66125714,
        0.66281703],
       ...,
       [0.59986413, 0.60144928, 0.59805254, ..., 0.        , 0.6699339 ,
        0.66531165],
       [0.66148571, 0.66011429, 0.66125714, ..., 0.6699339 , 0.        ,
        0.58627764],
       [0.66757246, 0.66530797, 0.66281703, ..., 0.66531165, 0.58627764,
        0.        ]])

# Consensus clustering

In [17]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [18]:
def scc_020(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.20,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_025(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.25,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_030(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.30,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_050(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.50,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )

In [19]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function __main__.scc_020(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_025(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_030(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_050(coassoc_distance_matrix, k, **kwargs)>}

In [20]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
            n_init=SC_N_INIT,
            random_state=next(RANDOM_STATES_ITER),
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/413 [00:00<?, ?it/s]

  0%|▏                                                            | 1/413 [00:27<3:06:28, 27.16s/it]

  0%|▎                                                            | 2/413 [00:28<2:12:32, 19.35s/it]

  1%|▍                                                            | 3/413 [00:30<1:36:48, 14.17s/it]

  1%|▌                                                            | 4/413 [00:58<2:05:59, 18.48s/it]

...

 99%|██████████████████████████████████████████████████████████▌| 410/413 [1:39:40<01:06, 22.04s/it]

100%|██████████████████████████████████████████████████████████▋| 411/413 [1:39:52<00:38, 19.10s/it]

100%|██████████████████████████████████████████████████████████▊| 412/413 [1:40:24<00:22, 22.94s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [1:40:44<00:00, 22.11s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [1:40:44<00:00, 14.64s/it]

In [21]:
consensus_results = pd.DataFrame(consensus_results)

In [22]:
display(consensus_results.shape)

(413, 12)

In [23]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,scc_050,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.163591,0.087346,0.207059,0.198572,0.168315,0.129355,0.201697,0.172070,0.129167
1,scc_050,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.169278,0.091581,0.209279,0.246141,0.224754,0.130349,0.250372,0.229236,0.129935
2,scc_050,"[2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, ...",4,0.155084,0.081946,0.187311,0.248798,0.211920,0.132093,0.254340,0.219736,0.131708
3,scc_050,"[4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 4, 3, ...",5,0.095293,0.063234,0.087100,0.214761,0.221643,0.105779,0.221310,0.230079,0.105815
4,scc_050,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, ...",6,0.098073,0.063326,0.092029,0.224145,0.232867,0.115376,0.231692,0.241950,0.115587


## Testing

In [24]:
assert not consensus_results.isna().any().any()

In [25]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       3
2       4
3       5
4       6
       ..
408    56
409    57
410    58
411    59
412    60
Name: partition, Length: 413, dtype: int64

## Save

In [26]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [27]:
consensus_results.to_pickle(output_file)